In [56]:
import PIL
import io
from __future__ import (absolute_import, division, print_function)
import os
import plotly.express as px
import folium
from branca.utilities import split_six
from folium.plugins import TimeSliderChoropleth
import numpy as np
import pandas as pd
import geopandas as gpd
from geopandas import GeoSeries, GeoDataFrame
import json
from urllib.request import urlopen
from shapely.geometry import Point
from shapely.geometry import Polygon


**Importing Data**

In [57]:
#names of the csv files to be read
csv_file_list = ["pp-2021.csv", "pp-2020.csv"]

#new list to store read csv files into
list_of_dataframes = []

#reading of each file in csv_file_list and storage in list_of_dataframes with column headers
for filename in csv_file_list:
    list_of_dataframes.append(pd.read_csv(filename, names = ['ID','Price', 
                                                             'Date of Transfer', 'Post Code',
                                                             'Property Type', 'Old/New','Duration','Address 1',
                                                             'Address 2', 'Address 3', 'Town/City', 'District',
                                                             'Borough', 'Region', 'PPDCategoryType', 'RecordStatus']))

#merging of each yearly csv into one data frame
merged_df = pd.concat(list_of_dataframes)

**Pre Processing 1**

In [58]:
#removal of duplicated entry
merged_df = merged_df.drop_duplicates(subset=['ID'], keep="first")

#removal of unnecessary columns
merged_df.drop(['ID','Address 1', 'Address 2', 'Address 3','Borough', 
         'Region', 'PPDCategoryType', 'RecordStatus'], axis=1, inplace=True)

**Extracting Poscode Suffix from Postcodes**

In [59]:
#Import regex function for reading postcodes
import re
regex = "^(?:(?P<a1>[Gg][Ii][Rr])(?P<d1>) (?P<s1>0)(?P<u1>[Aa]{2}))|(?:(?:(?:(?P<a2>[A-Za-z])(?P<d2>[0-9]{1,2}))|(?:(?:(?P<a3>[A-Za-z][A-Ha-hJ-Yj-y])(?P<d3>[0-9]{1,2}))|(?:(?:(?P<a4>[A-Za-z])(?P<d4>[0-9][A-Za-z]))|(?:(?P<a5>[A-Za-z][A-Ha-hJ-Yj-y])(?P<d5>[0-9]?[A-Za-z]))))) (?P<s2>[0-9])(?P<u2>[A-Za-z]{2}))$"


pc_parts = merged_df['Post Code'].str.extract(regex, re.IGNORECASE, expand=True)
merged_df['Postcode Area'] = pd.concat([pc_parts[['a%s'%i]] for i in range(1,6)], axis=1).bfill(axis=1).iloc[:,0]


**Pre Processing 2**

In [60]:
#removal of outliers
clean_df = merged_df.loc[(merged_df['Price'] <= (3000000)) & (merged_df['Price'] >= (10000)),]

#removal of data from incomplete months
clean_df = clean_df[clean_df['Date of Transfer']< '2021-04-01']

**Matching records to Regions**

In [62]:
#Mapping Postcode areas to a UK region
postcode_regions = pd.read_csv('postcode-areas.csv')
postcode_regions.drop('Postcode Area Name', axis=1, inplace=True)

clean_df2 = pd.merge(clean_df, 
                     postcode_regions, 
                     on ='Postcode Area', 
                     how ='left')

#Standardising with official region spellings
clean_df2["Region"].replace({"EastEngland": "East of England", 
                            "Yorkshire and the Humber": "Yorkshire and The Humber",
                            "Greater London": "London",
                            "West Midlands": "West Midlands (England)",
                            "South East": "South East (England)",
                            "South West": "South West (England)",
                            "East Midlands": "East Midlands (England)",
                            "North West": "North West (England)",
                            "North East": "North East (England)"
                           }, inplace=True)

#Removing excess regions
clean_df2.dropna(subset = ["Region"], inplace=True)
clean_df2 = clean_df2[clean_df2.Region != 'Scotland']



**Formatting date column**

In [63]:
#Convert to datetime
clean_df2['Date of Transfer'] = pd.to_datetime(clean_df2['Date of Transfer'])
clean_df2['Date of Transfer'] = clean_df2['Date of Transfer'].apply(lambda x: x.strftime('%Y%m%d'))

#Convert datetime to Epoch time
clean_df2['ModifiedDateTime'] = pd.Series(pd.to_numeric(clean_df2['Date of Transfer'], errors='coerce'),
                                                   dtype='int64')
clean_df2['ModifiedDateTime'] = pd.to_datetime(clean_df2['ModifiedDateTime'].astype('int64').astype('str'), 
                                              yearfirst=True)

#Create Year and Month columns for grouping
clean_df2['Year'] = clean_df2['ModifiedDateTime'].dt.year
clean_df2['Month'] = clean_df2['ModifiedDateTime'].dt.month


**GeoJSON file preparation for mapping**

In [64]:
#Link to geojson file
path_geojson = 'http://geoportal1-ons.opendata.arcgis.com/datasets/01fd6b2d7600446d8af768005992f76a_4.geojson'

#read geojson the file
gdf = gpd.read_file(path_geojson)

#Region IDs
region_id_map = {'North East (England)': 1,
                 'North West (England)': 2,
                 'Yorkshire and The Humber': 3,
                 'East Midlands (England)': 4,
                 'West Midlands (England)': 5,
                 'East of England': 6,
                 'London': 7,
                 'South East (England)': 8,
                 'South West (England)': 9,
                 'Wales': 10,
                 'Scotland': 11,
                 'Northern Ireland': 12}

# create a numerical id for each region
gdf["id"] = gdf["nuts118nm"].map(region_id_map)

# extract the id, geometry and regions
gdf = gdf[["id", "st_lengthshape","st_areashape","geometry","nuts118nm"]]

#renaming columns
gdf = gdf.rename(columns={"nuts118nm": "Region"})


**Median Price dataframe preparation for mapping**

In [65]:
#Calculate median monthly prices
df2 = clean_df2.groupby(['Region',(clean_df2["ModifiedDateTime"].dt.year),
                        (clean_df2["ModifiedDateTime"].dt.month)]).median()
df2.reset_index(level=0, inplace=True)

#Realign data column for folium
df2['Date'] = pd.to_datetime(df2[['Year', 'Month']].assign(DAY=1))
df2 = df2.set_index('Date').drop(['Year', 'Month'], axis=1)
df2.reset_index(level=0, inplace=True)
df2["DATE_for_Folium"] = (df2["Date"].astype(int)// 10**9).astype('U10')

#Map same unique ID as usef for the geoJSON
df2["id"] = df2["Region"].map(region_id_map)

**Creating colour palette for mapping**

In [77]:
#https://www.kaggle.com/imoore/easy-tutorial-for-plotting-with-python/notebook
def get_hex_colors_2(value, cats):
    
    if value == 0:
        return "#FFFFFF"
    elif value in cats[0]:
        return "#f7fcfd"
    elif value in cats[1]:
        return "#e0ecf4"
    elif value in cats[2]:
        return "#bfd3e6"
    elif value in cats[3]:
        return "#9ebcda"
    elif value in cats[4]:
        return "#8c96c6"
    elif value in cats[5]:
        return "#8c6bb1"
    elif value in cats[6]:
        return "#88419d"
    elif value in cats[7]:
        return "#810f7c"
    elif value in cats[8]:
        return "#4d004b"
    else:
        return "#4d004b"
    
    

**Median Price Style Dictionary Creation**

In [78]:
#https://www.kaggle.com/imoore/easy-tutorial-for-plotting-with-python/notebook
# selecting the column to be plotted
data_to_color = "Price"

#Splitting the data into 10 bins
cats, bins =  pd.qcut(df2[data_to_color].unique()[np.argsort(df2[data_to_color].unique())],
                      q = 9, retbins = True)

#Getting unique categories
cats = cats.unique()

#Assigning color to median price data
df2["COLORS"] = df2[data_to_color].apply(get_hex_colors_2, args = [cats]) 

#Values iterated in order to create the styledict
ccaas = list(df2["id"].unique())
dates = list(df2["DATE_for_Folium"].unique())

#Creating the style dictionary for the map
styledict = {}
for ccaa in ccaas:
    styledict[str(ccaa)] = {date: {'color': df2[(df2["id"] == ccaa) & 
                                                (df2["DATE_for_Folium"] == date)]["COLORS"].values[0],
                                   'opacity': 0.8} for date in dates}

**Median Price Map**

In [126]:
#https://www.kaggle.com/imoore/easy-tutorial-for-plotting-with-python/notebook
n = folium.Map(location=(52.5, 0), tiles='OpenStreetMap', zoom_start=5.5)

g = TimeSliderChoropleth(
    gdf.set_index("id").to_json(),
    styledict = styledict, overlay=True 
    ).add_to(n)

#https://geohackweek.github.io/ghw2018_web_portal_inlandwater_co2/InteractiveTimeSeries.html
#Adding Legend
state_geo = 'http://geoportal1-ons.opendata.arcgis.com/datasets/01fd6b2d7600446d8af768005992f76a_4.geojson'
n.choropleth(
    geo_data=state_geo,
    data=df2,
    columns=['Region', 'Price'],
    key_on='feature.properties.nuts118nm',
    fill_color= 'BuPu',
    fill_opacity=0,
    line_opacity=0,
    legend_name="Median Price (£)",
    smooth_factor=0,
    Highlight= True,
    line_color = "#0000",
    show=True,
    overlay=True,
)
folium.LayerControl().add_to(n)

#https://towardsdatascience.com/how-to-step-up-your-folium-choropleth-map-skills-17cf6de7c6fe
# Add hover functionality.
style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.2, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.0, 
                                'weight': 0.1}
NIL = folium.features.GeoJson(
    data = gdf,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['Region'],
        aliases=['Region:'],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)
n.add_child(NIL)
n.keep_in_front(NIL)

#Saving as html
n.save('TimeSeries_Median.html')

In [127]:
n